In [1]:
import numpy as np
from scipy.optimize import least_squares
from itertools import combinations

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 监测设备的坐标（经度，纬度，高程）
coords = np.array([
    [110.241, 27.204, 824],
    [110.780, 27.456, 727],
    [110.712, 27.785, 742],
    [110.251, 27.825, 850],
    [110.524, 27.617, 786],
    [110.467, 27.921, 678],
    [110.047, 27.121, 575]
])

# 检查经纬度范围
assert np.all((-180 <= coords[:, 0]) & (coords[:, 0] <= 180)), "经度范围应在-180°到180°之间"
assert np.all((-90 <= coords[:, 1]) & (coords[:, 1] <= 90)), "纬度范围应在-90°到90°之间"

# 音爆抵达时间
t_i = np.array([
    100.767,
    112.220,
    188.020,
    258.985,
    118.443,
    266.871,
    163.024
])

# 设备组合数量
device_combinations = list(combinations(range(len(coords)), 3))

# 初始化结果列表
results = []

# 对每个组合运行最小二乘法
for comb in device_combinations:
    comb_coords = coords[list(comb)]
    comb_t_i = t_i[list(comb)]
    
    # 初始猜测值 (x, y, z, t0)
    initial_guess = np.array([np.mean(comb_coords[:, 0]), np.mean(comb_coords[:, 1]), np.mean(comb_coords[:, 2]), np.min(comb_t_i)])
    
    # 使用最小二乘法进行数值优化求解，并设置音爆时间的非负约束
    bounds = ([-180, -90, 0, -np.inf], [180, 90, 50000, np.inf])  # 设置经度、纬度、高度和时间的范围
    result = least_squares(residuals, initial_guess, bounds=bounds, args=(v, comb_t_i, comb_coords))
    
    # 输出结果
    x_sol, y_sol, z_sol, t0_sol = result.x
    
    # 计算误差之和
    errors = comb_t_i - np.array([t0_sol + np.sqrt((lat_lon_to_length(x_sol, y_sol, comb_coords[i][0], comb_coords[i][1]))[0]**2 + (lat_lon_to_length(x_sol, y_sol, comb_coords[i][0], comb_coords[i][1]))[1]**2 + (z_sol - comb_coords[i][2])**2) / v for i in range(len(comb_t_i))])
    error_sum = np.sum(errors ** 2)
    
    results.append(((x_sol, y_sol, z_sol), t0_sol, error_sum, comb))

# 按误差之和排序
results.sort(key=lambda x: x[2])

# 输出前4的方案
print("前4的方案:")
for i, result in enumerate(results[:4]):
    coords_sol, t0_sol, error_sum, comb = result
    comb_str = ", ".join([f"设备{j+1}" for j in comb])
    print(f"方案 {i+1}:")
    print(f"  设备组合: {comb_str}")
    print(f"  音爆发生的位置: 经度 = {coords_sol[0]:.6f}, 纬度 = {coords_sol[1]:.6f}, 高程 = {coords_sol[2]:.3f} 米")
    print(f"  音爆发生的时间: t0 = {t0_sol:.3f} 秒")
    print(f"  误差之和: {error_sum:.3f}")
    print()

前4的方案:
方案 1:
  设备组合: 设备1, 设备4, 设备6
  音爆发生的位置: 经度 = 110.789071, 纬度 = 26.860486, 高程 = 704.902 米
  音爆发生的时间: t0 = -92.208 秒
  误差之和: 0.000

方案 2:
  设备组合: 设备1, 设备3, 设备4
  音爆发生的位置: 经度 = 111.424811, 纬度 = 26.175710, 高程 = 811.797 米
  音爆发生的时间: t0 = -376.741 秒
  误差之和: 0.000

方案 3:
  设备组合: 设备1, 设备2, 设备6
  音爆发生的位置: 经度 = 110.727112, 纬度 = 26.895969, 高程 = 22886.136 米
  音爆发生的时间: t0 = -82.879 秒
  误差之和: 0.000

方案 4:
  设备组合: 设备3, 设备4, 设备7
  音爆发生的位置: 经度 = 111.057391, 纬度 = 26.822051, 高程 = 731.018 米
  音爆发生的时间: t0 = -142.239 秒
  误差之和: 0.000

